# Fill in missing values: a case study with DFB data

A data science job may seem a glamourous one for very smart guys with eyeglasses and little bit nerds, but it is not. Among data scientist tasks there are quite a few that are "dirty", for example data cleaning. In particular, they must deal with missing data day in and day out, which becomes a painful but very important duty. Just remember that may machine learning algorithms do not know what to do with missing data.

So, in this notebook we are going to carry out some data cleaning tasks in a small dataset that was previously retrieved from the Deutscher Fußball Bund's archives. This dataset contains some personal information about players that manage to score at least a goal in 1.Bundesliga.

Firstly, let us import the `pandas` library and load the data. In order to reproduce the next steps locally you may download the CSV file here, and remember to change `SCORERS_PATH` to your local file.

In [2]:
# imports
import pandas as pd

# load data
SCORERS_PATH = '/mnt/287A29DF7A29AA90/PythonProjects/projects_repo/BuLi_scorers/data/spielers.csv'
df_spielers = pd.read_csv(SCORERS_PATH)

Once we have loaded the data, it is mandatory to make some explorations to gain insight about the data and the undetaking we got ahead.

In [3]:
df_spielers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840 entries, 0 to 3839
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Spieler      3840 non-null   object
 1   DFB_id       3840 non-null   int64 
 2   link         3840 non-null   object
 3   Nation       3826 non-null   object
 4   Nationalelf  3826 non-null   object
dtypes: int64(1), object(4)
memory usage: 150.1+ KB


Notice there are some missing values in columns `Nation` and `Nationalelf`, 14 to be exact. In addition, there is no need of the column `DFB_id` and `link` since they are OK by construction then, we can drop them to focus in the other features. Before we dive into the missing values question, let us take a look to our dataframe.

In [4]:
df_spielers.drop(columns=['DFB_id', 'link'], inplace=True)
df_spielers.head(15)

,Spieler,Nation,Nationalelf
0,Gerd Müller,['Deutschland'],Deutschland
1,Robert Lewandowski,['Polen'],Polen
2,Klaus Fischer,['Deutschland'],Deutschland
3,Jupp Heynckes,['Deutschland'],Deutschland
4,Manfred Burgsmüller,['Deutschland'],Deutschland
5,Claudio Pizarro,"['Peru', 'Italien']",Peru
6,Ulf Kirsten,['Deutschland'],Deutschland
7,Stefan Kuntz,['Deutschland'],Deutschland
8,Dieter Müller,['Deutschland'],Deutschland
9,Klaus Allofs,['Deutschland'],Deutschland


After a visual inspection we infer that `Nation` has lists of strings as values, whilst `Nationalelf` has strings. Recall that `Nation` stores which countries a footballer may represent and `Nationalelf` the chosen team. 

Regarding null values, we have two choices to fill the missing data: we can try to figure out an automatized method, perhaps relying on Transfermarkt, or we can do it by hand. Taking into account that we only have 14 "problematic" rows, it is feasible to complete the information by hand.

Let us take a closer look to the rows we are interested in.

In [5]:
mask_null_values = df_spielers['Nation'].isnull()
df_spielers[mask_null_values]

,Spieler,Nation,Nationalelf
1763,Boris Vukčević,NaN,NaN
2081,Bruma,NaN,NaN
2555,Takashi Usami,NaN,NaN
2838,Prince Tagoe,NaN,NaN
3286,Thanos Petsos,NaN,NaN
3331,Iver Fossum,NaN,NaN
3333,Franco Zuculini,NaN,NaN
3462,Tommy Svindal Larsen,NaN,NaN
3483,Jacopo Sala,NaN,NaN
3556,Babacar N'Diaye,NaN,NaN


Let us follow a naive approach: we will construct a couple of lists with the missing data first, and then fill the blanks. The information regarding FIFA eligibility and national teams was retrieved from the specialized portal Transfermarkt.

In [6]:
nation_lst = [
    ['Deutschland', 'Kroatien'], ['Portugal', 'Guinea-Bissau'], ['Japan'], ['Ghana'], 
    ['Griechenland', 'Deutschland'], ['Norway'], ['Argentinien', 'Italien'], ['Norway'], ['Italien'], ['Senegal', 'Deutschland'], ['Tunesien'], ['Japan'], ['Schweiz', 'Kosovo'], ['Ungarn']
]
df_spielers[mask_null_values]['Nation'] = nation_lst

/tmp/ipykernel_43970/857646728.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_spielers[mask_null_values]['Nation'] = nation_lst


What happened here? `Pandas` has issued a warning message because we have intended to do something that it may result in unwanted behavior. The essence is that the result of the command `df_spielers[mask_null_values]['Nation']` is not a "hard copy" of the dataframe's section but a "shallow copy". In other words, we are not watching the real data but an image. A deeper explanation for those interested may be find [here](https://realpython.com/pandas-settingwithcopywarning/).

Then, to avoid this warning we are going to use the `.loc` method to access the real data and fill in the missing values, but first let us construct a list for `Nationalelf` and after we will use a loop to complete the task.

In [7]:
nationalelf_lst = [
    'Deutschland', 'Portugal', 'Japan', 'Ghana', 'Griechenland', 'Norway', 'Argentina', 'Norway', 'Italien', 'Senegal',
     'Tunesien', 'Japan', 'Schweiz', 'Ungarn'
]

In [8]:
counter = 0
for k in range(len(mask_null_values)):
    if mask_null_values[k]:
        df_spielers.loc[k, 'Nation'] = nation_lst[counter]
        df_spielers.loc[k, 'Nationalelf'] = nationalelf_lst[counter]
        counter+=1

Let us check if the job was done correctly:

In [9]:
df_spielers[mask_null_values]

,Spieler,Nation,Nationalelf
1763,Boris Vukčević,"[Deutschland, Kroatien]",Deutschland
2081,Bruma,"[Portugal, Guinea-Bissau]",Portugal
2555,Takashi Usami,[Japan],Japan
2838,Prince Tagoe,[Ghana],Ghana
3286,Thanos Petsos,"[Griechenland, Deutschland]",Griechenland
3331,Iver Fossum,[Norway],Norway
3333,Franco Zuculini,"[Argentinien, Italien]",Argentina
3462,Tommy Svindal Larsen,[Norway],Norway
3483,Jacopo Sala,[Italien],Italien
3556,Babacar N'Diaye,"[Senegal, Deutschland]",Senegal


But, there is something interesting in how this data looks like and how the first 15 rows looks like. Notice how in the latter there are single quotation marks in the column `Nation`. Are the same data types stored in the aforementiones column?

In [10]:
type(df_spielers['Nation'][0])

str

Thus, `pandas` loaded the values in the column `Nation` as strings and not as lists. Consequently, we would need some transformation in order to bring the lists back. The fastest way to accomplish this is to use the function `literal_eval` from the module `ast` which is more efficient than rely on regular expressions.

In [11]:
from ast import literal_eval

df_spielers['Nation'] = df_spielers['Nation'].apply(
    lambda x: literal_eval(x) if isinstance(x, str) else x
)

The condition `isinstance(x, str)` was added to avoid problems in execution time due to the fact that some rows already have lists as values.

In [12]:
df_spielers

,Spieler,Nation,Nationalelf
0,Gerd Müller,[Deutschland],Deutschland
1,Robert Lewandowski,[Polen],Polen
2,Klaus Fischer,[Deutschland],Deutschland
3,Jupp Heynckes,[Deutschland],Deutschland
4,Manfred Burgsmüller,[Deutschland],Deutschland
...,...,...,...
3835,Achim Melcher,[Deutschland],Deutschland
3836,Marcelo Saracchi,[Uruguay],Uruguay
3837,Romeo Castelen,"[Niederlande, Suriname]",Niederlande
3838,Maicon,[Brasilien],Brasilien


Since I consider myself a perfectionist (when I can allow such luxury, of course) there is still one last step: to check if there exist another kind of errors in the data. An in depth revision threw that there were four players with wrong information, they are Raúl Bobadilla, Sébastien Haller, Pál Dardai and Roman Neustädter.

In [13]:
mask_wrongs = [False]*len(df_spielers)
wrong_pos = [526, 632, 871, 1602]
for pos in wrong_pos:
    mask_wrongs[pos] = True

df_spielers[mask_wrongs]

,Spieler,Nation,Nationalelf
526,Raúl Bobadilla,"[Argentinien, Paraguay]",Argentinien
632,Sébastien Haller,"[Frankreich, Elfenbeinküste (Côte d'Ivoire)]",Frankreich
871,Pál Dardai,"[Deutschland, Ungarn]",Deutschland
1602,Roman Neustädter,"[Deutschland, Russland]",Deutschland


In [14]:
df_spielers.iloc[526, 2] = 'Paraguay'
df_spielers.iloc[632, 2] = "Elfenbeinküste (Côte d'Ivoire)"
df_spielers.iloc[871, 2] = 'Ungarn'
df_spielers.iloc[871, 1] = ['Ungarn']
df_spielers.iloc[1602, 2] = 'Russland'

In [15]:
df_spielers[mask_wrongs]

,Spieler,Nation,Nationalelf
526,Raúl Bobadilla,"[Argentinien, Paraguay]",Paraguay
632,Sébastien Haller,"[Frankreich, Elfenbeinküste (Côte d'Ivoire)]",Elfenbeinküste (Côte d'Ivoire)
871,Pál Dardai,[Ungarn],Ungarn
1602,Roman Neustädter,"[Deutschland, Russland]",Russland


In [ ]:
df_spielers.to_csv('/mnt/287A29DF7A29AA90/PythonProjects/projects_repo/BuLi_scorers/data/spielers_fixed.csv')